#### 考察原先的fasta数据（CD-HIT之后与之前的效果）

In [60]:
import csv
import time
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn import model_selection, preprocessing, cross_validation, metrics
from sklearn import ensemble
from xgboost import XGBClassifier

from imblearn import combine
from imblearn import ensemble
from imblearn import over_sampling
from imblearn import under_sampling

%matplotlib inline
from matplotlib import pyplot
from collections import Counter

import MySQLdb
from sqlalchemy import create_engine


def draw(df, feature, isPrint=False, isTen=True):
    letter_counts = Counter(df[feature].values)
    print 'Total number:', len(letter_counts)
    lc_dict = dict(letter_counts)
    lc_dict = sorted(lc_dict.iteritems(), key=lambda d:d[1], reverse = True)
    df = pd.DataFrame.from_dict(lc_dict)
    df.columns = ['name', 'count']
    df.plot(x='name', y='count', kind='bar')
    if isPrint:
        if isTen:
            return df[:10]
        else:
            return df

In [3]:
name = []
value = []
count = 0
RNA_all = open('data/RNA-all-long-CD-HIT.fasta')
for line in RNA_all:
    if line[0] == '>':
        count += 1
        label = ','.join(line.strip("\r\n").split(',')[1])
        name_label = '>seq'+str(count)+','+label
        name.append(name_label.split(','))
    else:
        value.append('\n'+line.strip("\r\n"))
RNA_all.close()

In [4]:
name_list = [line.strip() for line in open('data/labels-39-names.txt')][:39]
name_list.insert(0, 'seq_name')
RNA_all_df = pd.concat([pd.DataFrame(np.array(name), columns=name_list), pd.DataFrame(np.array(value), columns=['seq_value'])], axis=1)
RNA_all_df[0:5]

,seq_name,Anterior,Apical,Axon,Basal,Cell body,Cell cortex,Cell junction,Cell leading edge,Cellular bud,...,Nucleus,Perinuclear,Peroxisome,Posterior,Pseudopodium,Ribosome,Somatodendritic compartment,Synapse,Vegetal,seq_value
0,>seq1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,\nGCUGAGUCAUCACUAGAGAGUGGGAAGGGCAGCAGCAGCAGAGA...
1,>seq2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,\nAUAAAGCUGAGACGCACCAGUCACCGACACAACUCCAGCCAUGG...
2,>seq3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,\nCGCGACCUCAGAUCAGACGUGGCGACCCGCUGAAUUUAAGCAUA...
3,>seq4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,\nAGGCUUACGUUUACUUUGCGCCGGAAAGAGAACCUGUAAACGCU...
4,>seq5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,\nAAGAUUUAGGCAUCCUCUUACACUGCUGGGAAAUAUCAGUGUGA...


In [95]:
from matplotlib import pyplot
from collections import Counter
    
def get_new_df():
    df = RNA_all_df.loc[:, ['seq_name', 'Cytosol', 'Endoplasmic reticulum', 'Exosome', 'Nucleus', 'Ribosome', 'seq_value']]
    return df

def write_fasta(df, file_path):
    # df = pd.DataFrame([[1, '\nabs'], [1, '\nabs']])
    df.to_csv(file_path, quoting=csv.QUOTE_NONE, escapechar=' ',header=False, index=False)

def write_fea(df, file_path):
    df = df.loc[:, ['Cytosol', 'Endoplasmic reticulum', 'Exosome','Nucleus', 'Ribosome']]
    df.to_csv(file_path, quoting=csv.QUOTE_NONE, escapechar=' ',header=False, index=False)

write_fasta(get_new_df(), 'data/RNA-all-choosed.fasta')

In [26]:
!python /home01/shixiangwan/softwares/pse-in-one/Pse-in-One/pse.py data/RNA-all-long.fasta data/fea/RNA-all-long-Pse.csv RNA PC-PseDNC-General -f csv

In [47]:
len(labels)

33470

In [53]:
# feature = '/home01/shixiangwan/softwares/pse-in-one/Pse-in-One/data/RNA-all-choosed-out.csv'
# label_df = RNA_all_df.iloc[:, 1:40]
feature_df = pd.read_csv('data/fea/RNA-all-long-Pse.csv', header=None)
labels = [map(int, list(l.strip().split(',')[1])) for l in open("data/RNA-all-long.fasta") if l[0] == '>']
name_list = [line.strip() for line in open('data/labels-39-names.txt')][:39]
label_df = pd.DataFrame(labels, columns=name_list)

print feature_df.shape
print label_df.shape

(33470, 18)
(33470, 39)


In [55]:
label_list = ['Nucleus', 'Cytosol', 'Endoplasmic reticulum', 'Exosome', 'Ribosome', 'Extracellular vesicle', 'Cytoplasm', 'Nucleolus']
Counter(label_df['Cytoplasm'])

Counter({0: 24493, 1: 8977})

In [59]:
print time.ctime(), '交叉验证 ...'
clf = XGBClassifier() # n_estimators=600, subsample=0.7
# clf = ensemble.AdaBoostClassifier()
# clf = ensemble.RandomForestClassifier()

### train feature dataset
train = feature_df

for lab in label_list:
    label = label_df[lab]
    print time.ctime(), '>>', lab, dict(Counter(label))

    try:
        sm = SMOTE(random_state=42)
        train_res, label_res = sm.fit_sample(train.as_matrix(), map(int, label))

        # train_x, valid_x, train_y, valid_y = cross_validation.train_test_split(train_res, label_res, test_size=0.33, random_state=42)
        # forecast = clf.fit(train_x, train_y).predict(valid_x)
        # print 'f1_score', metrics.f1_score(valid_y, forecast)
        # print 'accuracy_score', metrics.accuracy_score(valid_y, forecast)
        # f1_score 0.653616456536
        # accuracy_score 0.629491615651

        X = train_res
        y = label_res
    #     kf = cross_validation.KFold(len(X), n_folds=5)
        kf = cross_validation.StratifiedKFold(y, n_folds=10)
        # print time.ctime(), '>>', clf.get_params
        count_num = 0
        forecasts = []
        forecasts2 = []
        for train_index, test_index in kf:
    #         count_num += 1
    #         print time.ctime(), '>> '+str(count_num)+' fold ...',

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            y_train = map(int, y_train)
            y_test = map(int, y_test)

            forecast = clf.fit(X_train, y_train).predict(X_test)
            score = metrics.f1_score(y_test, forecast)
            score2 = metrics.accuracy_score(y_test, forecast)
            forecasts.append(score)
            forecasts2.append(score2)
    #         print 'f1_score:', score, 'accuracy_score:',score2
        print time.ctime(), '>> cross-validate:', 'f1_score:', np.array(forecasts).mean(), 'accuracy_score:', np.array(forecasts2).mean()
    except Exception, e:  
        print time.ctime(), Exception, ":", e

Thu Jun  1 14:46:53 2017 交叉验证 ...
Thu Jun  1 14:46:53 2017 >> Nucleus {0: 21833, 1: 11637}
Thu Jun  1 14:48:23 2017 >> cross-validate: f1_score: 0.546328198805 accuracy_score: 0.508178844937
Thu Jun  1 14:48:23 2017 >> Cytosol {0: 25236, 1: 8234}
Thu Jun  1 14:49:54 2017 >> cross-validate: f1_score: 0.576441848788 accuracy_score: 0.575751846876
Thu Jun  1 14:49:54 2017 >> Endoplasmic reticulum {0: 30548, 1: 2922}
Thu Jun  1 14:51:24 2017 >> cross-validate: f1_score: 0.673392165048 accuracy_score: 0.637816675724
Thu Jun  1 14:51:24 2017 >> Exosome {0: 30622, 1: 2848}
Thu Jun  1 14:52:55 2017 >> cross-validate: f1_score: 0.670839876471 accuracy_score: 0.638266840504
Thu Jun  1 14:52:55 2017 >> Ribosome {0: 20145, 1: 13325}
Thu Jun  1 14:54:26 2017 >> cross-validate: f1_score: 0.537211812239 accuracy_score: 0.534313699882
Thu Jun  1 14:54:26 2017 >> Extracellular vesicle {0: 33470}


/usr/local/lib/python2.7/dist-packages/imblearn/base.py:100: UserWarning: Only one class detected, something will get wrong
  warnings.warn('Only one class detected, something will get wrong')


Thu Jun  1 14:55:09 2017 >> cross-validate: f1_score: 0.0 accuracy_score: 1.0
Thu Jun  1 14:55:09 2017 >> Cytoplasm {0: 24493, 1: 8977}
Thu Jun  1 14:56:38 2017 >> cross-validate: f1_score: 0.579436050494 accuracy_score: 0.523684735961
Thu Jun  1 14:56:38 2017 >> Nucleolus {0: 33461, 1: 9}
Thu Jun  1 14:58:05 2017 >> cross-validate: f1_score: 0.998910823394 accuracy_score: 0.998909212218
